In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gps-dataset/blackmesa.geojson
/kaggle/input/gps-dataset/Redv1.geojson
/kaggle/input/gps-dataset/Redv2.geojson


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gps-dataset/blackmesa.geojson
/kaggle/input/gps-dataset/Redv1.geojson
/kaggle/input/gps-dataset/Redv2.geojson


In [3]:
!pip install movingpandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 kB 2.8 MB/s eta 0:00:00


In [5]:
!pip install hvplot

In [6]:
# !pip install holoviews==1.18.0

In [7]:
# !pip install bokeh==3.3.1


In [8]:
# !conda install -c conda-forge holoviews=1.18.0 bokeh=3.3.1


In [9]:
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import hvplot.pandas 
import matplotlib.pyplot as plt

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim
from os.path import exists
from urllib.request import urlretrieve

import warnings
warnings.filterwarnings('ignore')

plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=300, frame_height=500))
hvplot_defaults = {'tiles':None, 'cmap':'Viridis', 'colorbar':True}

mpd.show_versions()


MovingPandas 0.17.2

SYSTEM INFO
-----------
python     : 3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]
executable : /opt/conda/bin/python3.10
machine    : Linux-5.15.133+-x86_64-with-glibc2.31

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : 3.11.1
GEOS lib   : /opt/conda/lib/libgeos_c.so
GDAL       : 3.6.4
GDAL data dir: /opt/conda/lib/python3.10/site-packages/fiona/gdal_data
PROJ       : 9.3.1
PROJ data dir: /opt/conda/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.14.2
pandas     : 2.2.0
fiona      : 1.9.5
numpy      : 1.26.3
shapely    : 1.8.5.post1
rtree      : 1.2.0
pyproj     : 3.6.1
matplotlib : 3.7.4
mapclassify: 2.6.1
geopy      : 2.4.1
holoviews  : 1.18.1
hvplot     : 0.9.2
geoviews   : 1.11.0
stonesoup  : None


In [46]:
df = gpd.read_file('/kaggle/input/gps-dataset/Redv2.geojson')

In [47]:
df

,GMT Time,Latitude,Longitude,Altitude,Duration,Temperatur,Voltage,DOP,Satellites,Cause of F,Lab_Coord,Lab_Dur,Lab_Sat,Lab_Alt,No,time,distance,speed,geometry
0,8/10/2021 14:31,34.790283,-108.305657,2690.83,65,19.0,0,3.0,5,GPS Schedule,0.0,0.0,0.0,0.0,1,8/10/2021 8:31,-1.000000,-1.000000,POINT (746520.733 3853095.300)
1,8/10/2021 14:45,34.795679,-108.303512,2690.13,3,18.0,0,3.6,5,GPS Schedule,0.0,0.0,0.0,0.0,2,8/10/2021 8:45,45.880294,3.276863,POINT (746700.929 3853699.248)
2,8/10/2021 15:00,34.792383,-108.297636,2685.92,3,18.5,0,3.8,5,GPS Schedule,0.0,0.0,0.0,0.0,3,8/10/2021 9:00,85.194239,2.353872,POINT (747248.551 3853347.996)
3,8/10/2021 15:15,34.794316,-108.304693,2677.39,3,18.5,0,3.2,5,GPS Schedule,0.0,0.0,0.0,0.0,4,8/10/2021 9:15,8.960635,0.463648,POINT (746596.977 3853545.118)
4,8/10/2021 15:30,34.799379,-108.307767,2651.16,2,18.5,0,2.6,5,GPS Schedule,0.0,0.0,0.0,0.0,5,8/10/2021 9:30,9.585296,0.702347,POINT (746300.609 3854099.176)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2712,10/8/2021 14:30,34.938247,-108.277773,1779.44,2,14.5,0,1.0,6,GPS Schedule,0.0,0.0,0.0,0.0,3540,10/8/2021 8:30,10.557870,0.495932,POINT (748626.601 3869579.002)
2713,10/8/2021 14:45,34.939316,-108.276832,1792.66,3,15.5,0,1.0,8,GPS Schedule,0.0,0.0,0.0,0.0,3541,10/8/2021 8:45,9.279421,14.271261,POINT (748709.310 3869699.891)
2714,10/8/2021 15:00,34.942255,-108.278813,1786.06,1,16.0,0,1.0,8,GPS Schedule,0.0,0.0,0.0,0.0,3542,10/8/2021 9:00,561.571028,7.467805,POINT (748519.479 3870021.045)
2715,10/8/2021 15:46,34.828757,-108.307587,2456.15,64,14.0,0,1.2,7,GPS Schedule,0.0,0.0,0.0,0.0,3545,10/8/2021 9:46,35.853340,0.494999,POINT (746229.565 3857358.763)


In [48]:
df['t'] = pd.to_datetime(df['time'])
df = df.set_index('t').tz_localize(None)

In [49]:
df.head()

,GMT Time,Latitude,Longitude,Altitude,Duration,Temperatur,Voltage,DOP,Satellites,Cause of F,Lab_Coord,Lab_Dur,Lab_Sat,Lab_Alt,No,time,distance,speed,geometry
t,,,,,,,,,,,,,,,,,,,
2021-08-10 08:31:00,8/10/2021 14:31,34.790283,-108.305657,2690.83,65,19.0,0,3.0,5,GPS Schedule,0.0,0.0,0.0,0.0,1,8/10/2021 8:31,-1.000000,-1.000000,POINT (746520.733 3853095.300)
2021-08-10 08:45:00,8/10/2021 14:45,34.795679,-108.303512,2690.13,3,18.0,0,3.6,5,GPS Schedule,0.0,0.0,0.0,0.0,2,8/10/2021 8:45,45.880294,3.276863,POINT (746700.929 3853699.248)
2021-08-10 09:00:00,8/10/2021 15:00,34.792383,-108.297636,2685.92,3,18.5,0,3.8,5,GPS Schedule,0.0,0.0,0.0,0.0,3,8/10/2021 9:00,85.194239,2.353872,POINT (747248.551 3853347.996)
2021-08-10 09:15:00,8/10/2021 15:15,34.794316,-108.304693,2677.39,3,18.5,0,3.2,5,GPS Schedule,0.0,0.0,0.0,0.0,4,8/10/2021 9:15,8.960635,0.463648,POINT (746596.977 3853545.118)
2021-08-10 09:30:00,8/10/2021 15:30,34.799379,-108.307767,2651.16,2,18.5,0,2.6,5,GPS Schedule,0.0,0.0,0.0,0.0,5,8/10/2021 9:30,9.585296,0.702347,POINT (746300.609 3854099.176)


In [50]:
original_crs = df.crs
original_crs

<Projected CRS: EPSG:26912>
Name: NAD83 / UTM zone 12N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 114°W and 108°W - onshore and offshore. Canada - Alberta; Northwest Territories; Nunavut; Saskatchewan. United States (USA) - Arizona; Colorado; Idaho; Montana; New Mexico; Utah; Wyoming.
- bounds: (-114.0, 31.33, -108.0, 84.0)
Coordinate Operation:
- name: UTM zone 12N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
!pip install geoviews holoviews cartopy

In [51]:
import hvplot.pandas

In [52]:
df = df.to_crs(epsg=4326)

In [53]:
one=df.hvplot(title='Geographic extent of the dataset', geo=True, tiles='OSM')

In [54]:
one

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

In [ ]:
%load_ext holoviews.extension
%matplotlib inline

In [55]:
df = df.to_crs({'init': 'epsg:4326'})
( df.hvplot(title='OSM showing paths and fences', size=2, geo=True, tiles='OSM') +
  df.hvplot(title='Imagery showing land cover details', size=2, color='red', geo=True, tiles='EsriImagery') )

:Layout
   .Overlay.I  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]

In [57]:
from pyproj import CRS
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim, Layout
from os.path import exists
from urllib.request import urlretrieve

In [58]:
temp = df.to_crs(CRS(25832))
temp['geometry'] = temp['geometry'].buffer(5)
total_area = temp.dissolve(by='No').area

In [59]:
print("The dataset covers the time between {} and {}.".format(df.index.min(), df.index.max()))

The dataset covers the time between 2021-08-10 08:31:00 and 2021-10-08 10:00:00.


In [60]:
print("That's {}".format(df.index.max() - df.index.min()))

That's 59 days 01:29:00


In [61]:
df['No'].resample('1d').count().hvplot(title='Number of records per day')

:Curve   [t]   (No)

In [62]:
df['Y-M'] = df.index.to_period('M')    
Layout([df[df['Y-M']==i].hvplot(title=str(i), size=2, geo=True, tiles='OSM') for i in df['Y-M'].unique()])

:Layout
   .Overlay.I   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.III :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]

In [63]:
t = df.reset_index().t
df = df.assign(delta_t=t.diff().values)
df['delta_t'] = df['delta_t'].dt.total_seconds()/60
pd.DataFrame(df).hvplot.hist('delta_t', title='Histogram of intervals between consecutive records (in minutes)', bins=60, bin_range=(0, 60))

:Histogram   [delta_t]   (delta_t_count)

In [64]:
df['CollarId']=6

In [65]:
tc = mpd.TrajectoryCollection(df, 'CollarId')
traj = tc.trajectories[0]
traj.add_speed(overwrite=True)
max_speed = traj.df.speed.max()
print("The highest computed speed is {:,.2f} m/s ({:,.2f} km/h)".format(max_speed, max_speed*3600/1000))

The highest computed speed is 4.66 m/s (16.78 km/h)


In [67]:
tc = mpd.TrajectoryCollection(df, 'CollarId')
traj = tc.trajectories[0]

In [68]:
pd.DataFrame(traj.df).hvplot.hist('speed', title='Histogram of speeds (in meters per minute)', bins=90)

:Histogram   [speed]   (speed_count)

In [69]:
traj.add_direction(overwrite=True)
pd.DataFrame(traj.df).hvplot.hist('direction', title='Histogram of directions', bins=90)

:Histogram   [direction]   (direction_count)

In [70]:
pd.DataFrame(traj.df).hvplot.heatmap(title='Mean speed by hour of day and month of year', 
                                     x='t.hour', y='t.month', C='speed', reduce_function=np.mean)

:HeatMap   [t.hour,t.month]   (speed)

In [71]:
traj.df['n'] = 6
pd.DataFrame(traj.df).hvplot.heatmap(title='Record count by temperature and month of year', 
                                     x='Temperatur', y='t.month', C='n', reduce_function=np.sum)

:HeatMap   [Temperatur,t.month]   (n)

In [72]:
pd.DataFrame(traj.df).hvplot.heatmap(title='Mean speed by temperature and month of year', 
                                     x='Temperatur', y='t.month', C='speed', reduce_function=np.mean)

:HeatMap   [Temperatur,t.month]   (speed)

In [73]:
traj.df['dir_class'] = ((traj.df['direction']-22.5)/45).round(0)
temp = traj.df
Layout([temp[temp['dir_class']==i].hvplot(geo=True, tiles='OSM', size=2, width=300, height=300, title=str(int(i*45))+"°") for i in sorted(temp['dir_class'].unique())])

:Layout
   .Overlay.I    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.III  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.IV   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.V    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VI   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VII  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VIII :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]

In [38]:
traj.df['speed_class'] = (traj.df['speed']*2).round(1)
temp = traj.df
plots = []
for i in sorted(temp['speed_class'].unique()):
    filtered = temp[temp['speed_class']==i]
    if len(filtered) < 20: 
        continue
    plots.append(filtered.hvplot(geo=True, tiles='EsriImagery', color='red', size=2, width=300, height=300, title=str(i/2)))
Layout(plots)

:Layout
   .Overlay.I     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.II    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.III   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.IV    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.V     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VI    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VII   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.VIII  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.IX    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.X     :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XI    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XII   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XIII  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XIV   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XV    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XVI   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XVII  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XVIII :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XIX   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]
   .Overlay.XX    :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Points.I :Points   [Longitude,Latitude]

In [74]:
daily = mpd.TemporalSplitter(tc).split(mode='day')

In [75]:
Layout([daily.trajectories[i].hvplot(title=daily.trajectories[i].id, c='speed', line_width=2, cmap='RdYlBu') for i in range(0,7)])

:Layout
   .Overlay.I   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.II  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.III :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.IV  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.V   :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.VI  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.VII :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)

In [76]:
daily_starts = daily.get_start_locations()
daily_starts.set_index(pd.to_datetime(daily_starts['time']), inplace=True) 
daily_starts['month'] = daily_starts.index.month
daily_starts.hvplot(c='month', geo=True, tiles='EsriImagery', cmap='autumn')

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (month)

In [77]:
moving = mpd.TrajectoryCollection(traj.df[traj.df['speed'] > 1.5], 'CollarId')
moving = mpd.ObservationGapSplitter(moving).split(gap=timedelta(minutes=70))

In [78]:
moving.get_start_locations().hvplot(c='month', geo=True, tiles='EsriImagery', color='red')

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

In [79]:
daily_lengths = [traj.get_length() for traj in daily]
daily_t = [traj.get_start_time() for traj in daily]

In [80]:
daily_lengths = pd.DataFrame(daily_lengths, index=daily_t, columns=['length'])
daily_lengths.hvplot(title='Daily trajectory length')

:Curve   [index]   (length)